## Modele de clasificare
Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. p=5%, p parametru) cu numpy.nan.

1. (20 puncte) Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

*Cerinta*: In cazul in care folositi un pachet ce trebuie instalat (nu face parte din distributia standard anaconda), includeti intr-o celula o comanda de instalare corespunzatoare folosind semn de exclamare, de exemplu:
```python
!pip install missingpy
```
(sursa: [https://github.com/epsilon-machine/missingpy](https://github.com/epsilon-machine/missingpy)). La executia celulei in Jupyter Notebook se instaleaza pachetul, iar in celulele ulterioare importurile din noul pachet functioneaza. 

In [1]:
import numpy as np

In [8]:
wine=np.genfromtxt("Datasets/wine.data", delimiter=",")
iris=np.genfromtxt("Datasets/iris.data", delimiter=",")
machine=np.genfromtxt("Datasets/machine.data", delimiter=",")
wdbc=np.genfromtxt("Datasets/wdbc.data", delimiter=",")
print(machine)

[[ nan  nan 125. ... 128. 198. 199.]
 [ nan  nan  29. ...  32. 269. 253.]
 [ nan  nan  29. ...  32. 220. 253.]
 ...
 [ nan  32. 125. ...  14.  52.  41.]
 [ nan  nan 480. ...   0.  67.  47.]
 [ nan  nan 480. ...   0.  45.  25.]]
